# Inferring enhancer-driven Gene Regulatory Networks (eGRNs) using SCENIC+

In [1]:
# Set up Environment
import dill
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas
import pyranges
# Set stderr to null to avoid strange messages from ray
import sys
_stderr = sys.stderr
null = open(os.devnull,'wb')

# set working directory
work_dir = '/g/scb/zaugg/deuner/SCENIC+/'
# set tmp directory
tmp_dir = '/g/scb/zaugg/deuner/SCENIC+/tmp/'
# set the figures directory
fig_dir = '/g/scb/zaugg/deuner/SCENIC+/figures/'
# set the output data directory
out_dir = '/g/scb/zaugg/deuner/SCENIC+/outputdata'

# Load the AnnData object containing the scRNA-seq side of the analysis
adata = sc.read_h5ad(os.path.join(work_dir, 'tmp/timecourse.nomicro.subset.adata.h5ad'))

# Load the cisTopic object containing the scATAC-seq side of the analysis.
cistopic_obj = dill.load(open(os.path.join(work_dir, 'tmp/scATAC/cistopic_obj.pkl'), 'rb'))

# Load the motif enrichment dictionary containing the motif enrichment results.
menr = dill.load(open(os.path.join(work_dir, 'tmp/motifs/menr.pkl'), 'rb'))

In [2]:
cistopic_obj.cell_data

,Total_nr_frag,Total_nr_frag_in_regions,Unique_nr_frag,cisTopic_nr_frag,FRIP,cisTopic_log_nr_frag,Log_total_nr_frag,cisTopic_log_nr_acc,Dupl_rate,TSS_enrichment,...,celltype,basic_celltype,pseudotime,pseudotime_clusters_n7,pseudotime_clusters_n14,wsnn_res.0.8,celltype_wnn,sampleID,barcode,sample_id
ATGATGACTAATGTGA_timecourse,754958,334165,5337,2928,0.475173,3.466571,5.877923,3.435048,0.992931,13.003266,...,neuron - development-2,neuron,49.537279,5,10,14,neuron - development-2,timecourse,ATGATGACTAATGTGA,timecourse
ACCTCATGAACGGTTG_timecourse,1068310,567645,7916,4830,0.554699,3.683947,6.028697,3.638689,0.992590,10.767352,...,diff,diff,33.233434,4,7,1,diff,timecourse,ACCTCATGAACGGTTG,timecourse
GGCTAGCACCCTCACG_timecourse,1010867,480271,7345,4395,0.459224,3.642959,6.004694,3.593729,0.992734,10.093255,...,diff,neuron,34.178498,4,7,1,diff,timecourse,GGCTAGCACCCTCACG,timecourse
ATGCTTATGCCCGTAG_timecourse,2812326,1312396,20128,11030,0.487629,4.042576,6.449066,3.981366,0.992843,8.875505,...,diff - NPC-like,diff,19.373090,2,4,2,diff - NPC-like,timecourse,ATGCTTATGCCCGTAG,timecourse
ACAGCACGAGCGCTAA_timecourse,1982065,911086,14680,7644,0.485150,3.883321,6.297118,3.829497,0.992594,9.621857,...,diff,diff,30.608254,4,7,1,diff,timecourse,ACAGCACGAGCGCTAA,timecourse
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CGTAAAGCTACTAACC_timecourse,306758,167007,2276,1410,0.499561,3.149219,5.486796,3.129045,0.992580,10.882922,...,neuron - excitatory,neuron,37.653173,4,8,6,neuron - excitatory,timecourse,CGTAAAGCTACTAACC,timecourse
GATATTGGACCCGGAA_timecourse,289259,160136,2164,1403,0.498614,3.147058,5.461287,3.123852,0.992519,10.631213,...,neuron - excitatory,neuron,34.608211,4,7,6,neuron - excitatory,timecourse,GATATTGGACCCGGAA,timecourse
CTGAGTGACAACAAAG_timecourse,562791,299255,4327,2559,0.477467,3.40807,5.750347,3.376029,0.992312,8.910482,...,diff - hiPSC-like,diff,7.033482,1,2,5,diff - hiPSC-like,timecourse,CTGAGTGACAACAAAG,timecourse
AAGCTCCTGGCGGAAC_timecourse,496358,202754,3849,1876,0.463757,3.273233,5.695795,3.261025,0.992246,11.987138,...,mature.neuron - adhesion,neuron,56.844036,6,12,7,mature.neuron - adhesion,timecourse,AAGCTCCTGGCGGAAC,timecourse


In [3]:
# adapt barcodes of cistopic object to timecourse_"barcode"
new_bcs = []
old_bcs = cistopic_obj.cell_names
for i in range(len(old_bcs)):
    split_bc = str.split(old_bcs[i], "_")
    new_bc = split_bc[1] + "_" + split_bc[0]
    new_bcs.append(new_bc)
    cistopic_obj.selected_model.cell_topic.columns.values[i] = new_bc

cistopic_obj.cell_names = new_bcs
cistopic_obj.cell_names

['timecourse_ATGATGACTAATGTGA',
 'timecourse_ACCTCATGAACGGTTG',
 'timecourse_GGCTAGCACCCTCACG',
 'timecourse_ATGCTTATGCCCGTAG',
 'timecourse_ACAGCACGAGCGCTAA',
 'timecourse_CGGGATTACATAGCTG',
 'timecourse_AACGGTTACCGCTAAG',
 'timecourse_ATGCTTTACGAGGGAT',
 'timecourse_AGGCCCAACCTCCTCT',
 'timecourse_ATGATGACTGGTCCAC',
 'timecourse_CCAAGGTACTCAATCC',
 'timecourse_CAATTTGCTTGTTGCG',
 'timecourse_CATGCTATGGTGCTGT',
 'timecourse_ACAGCCAGACCTAGCT',
 'timecourse_ACTAGGATGTGAGTAA',
 'timecourse_ACAAACACTTCGCTCC',
 'timecourse_ACAATAGCTTAGCTGC',
 'timecourse_AAGGTTTGACAGGCTG',
 'timecourse_AATGAGGGACAAGAAA',
 'timecourse_CGATCCTGACTAGTTT',
 'timecourse_ACGGCTCCTAGTAGCG',
 'timecourse_ACATTAGGAAGTAAGG',
 'timecourse_AAGCCTAACTTTGTGC',
 'timecourse_GAGCCTGACCTAGCTT',
 'timecourse_AACGTAACTGTGACCC',
 'timecourse_AGGTTACACGTTTACT',
 'timecourse_AAGGAAGCTGGGACAT',
 'timecourse_AGCCTTGTGCATCCCT',
 'timecourse_CAATACCTGCTTGCTC',
 'timecourse_ATAATGCGATTATTGC',
 'timecourse_AACCCTCACGGTAACT',
 'timeco

In [4]:
cistopic_obj.cell_data.index = new_bcs

In [5]:
cistopic_obj.selected_model.cell_topic

,timecourse_ATGATGACTAATGTGA,timecourse_ACCTCATGAACGGTTG,timecourse_GGCTAGCACCCTCACG,timecourse_ATGCTTATGCCCGTAG,timecourse_ACAGCACGAGCGCTAA,timecourse_CGGGATTACATAGCTG,timecourse_AACGGTTACCGCTAAG,timecourse_ATGCTTTACGAGGGAT,timecourse_AGGCCCAACCTCCTCT,timecourse_ATGATGACTGGTCCAC,...,timecourse_AACAGCTACTACGGAC,timecourse_GCGGTTGACCTTACCA,timecourse_AGCCGGTACAATTGCG,timecourse_AGGGATTTGCATTAGC,timecourse_CTTAGAAACGCAATAC,timecourse_CGTAAAGCTACTAACC,timecourse_GATATTGGACCCGGAA,timecourse_CTGAGTGACAACAAAG,timecourse_AAGCTCCTGGCGGAAC,timecourse_ACATTAGGATTAGCGA
Topic1,0.013749,0.002527,0.006826,0.004686,0.010014,0.007023,0.065968,0.002975,0.016546,0.098633,...,0.016346,0.010570,0.011662,0.001997,0.003034,0.015849,0.014583,0.027658,0.046491,0.007009
Topic2,0.226875,0.003663,0.009342,0.004167,0.001635,0.005131,0.001635,0.015227,0.050591,0.002664,...,0.177885,0.197851,0.241423,0.065131,0.017836,0.096794,0.042120,0.002936,0.027281,0.082153
Topic3,0.015912,0.025471,0.012865,0.046327,0.059845,0.021023,0.015801,0.052123,0.027979,0.006309,...,0.016799,0.016160,0.008577,0.036682,0.052176,0.010835,0.013134,0.008704,0.030483,0.027818
Topic4,0.021682,0.011387,0.024943,0.018601,0.020892,0.006456,0.061547,0.002693,0.002064,0.048385,...,0.030826,0.021052,0.029395,0.028108,0.035598,0.014416,0.010960,0.011176,0.017676,0.017413
Topic5,0.019158,0.004345,0.007581,0.009255,0.007956,0.003618,0.099262,0.000581,0.011719,0.098081,...,0.005034,0.021052,0.019372,0.009012,0.017836,0.006537,0.016033,0.120777,0.050760,0.004697
Topic6,0.013027,0.042736,0.025447,0.054426,0.032651,0.021401,0.010117,0.044659,0.007146,0.021107,...,0.020419,0.026642,0.009348,0.011740,0.014876,0.032324,0.046467,0.008292,0.012340,0.014523
Topic7,0.002569,0.021382,0.019911,0.089213,0.075132,0.022914,0.004252,0.056348,0.007654,0.005425,...,0.020871,0.003581,0.006264,0.009402,0.036782,0.023729,0.030525,0.004584,0.024080,0.008743
Topic8,0.005094,0.005480,0.005567,0.009566,0.024566,0.019131,0.071562,0.006355,0.007908,0.077761,...,0.013631,0.016160,0.013975,0.011350,0.005403,0.012267,0.009511,0.113772,0.057164,0.008165
Topic9,0.019158,0.012977,0.015633,0.065849,0.049408,0.042211,0.020042,0.029873,0.021627,0.031378,...,0.031731,0.050402,0.010120,0.027718,0.036190,0.012267,0.016757,0.009116,0.060366,0.015101
Topic10,0.009782,0.009342,0.002799,0.015070,0.001929,0.012320,0.080585,0.001848,0.007400,0.106474,...,0.005939,0.010570,0.012433,0.008622,0.026717,0.026594,0.005163,0.049907,0.042223,0.011055


In [6]:
cistopic_obj.cell_data
list(cistopic_obj.cell_names.copy())

['timecourse_ATGATGACTAATGTGA',
 'timecourse_ACCTCATGAACGGTTG',
 'timecourse_GGCTAGCACCCTCACG',
 'timecourse_ATGCTTATGCCCGTAG',
 'timecourse_ACAGCACGAGCGCTAA',
 'timecourse_CGGGATTACATAGCTG',
 'timecourse_AACGGTTACCGCTAAG',
 'timecourse_ATGCTTTACGAGGGAT',
 'timecourse_AGGCCCAACCTCCTCT',
 'timecourse_ATGATGACTGGTCCAC',
 'timecourse_CCAAGGTACTCAATCC',
 'timecourse_CAATTTGCTTGTTGCG',
 'timecourse_CATGCTATGGTGCTGT',
 'timecourse_ACAGCCAGACCTAGCT',
 'timecourse_ACTAGGATGTGAGTAA',
 'timecourse_ACAAACACTTCGCTCC',
 'timecourse_ACAATAGCTTAGCTGC',
 'timecourse_AAGGTTTGACAGGCTG',
 'timecourse_AATGAGGGACAAGAAA',
 'timecourse_CGATCCTGACTAGTTT',
 'timecourse_ACGGCTCCTAGTAGCG',
 'timecourse_ACATTAGGAAGTAAGG',
 'timecourse_AAGCCTAACTTTGTGC',
 'timecourse_GAGCCTGACCTAGCTT',
 'timecourse_AACGTAACTGTGACCC',
 'timecourse_AGGTTACACGTTTACT',
 'timecourse_AAGGAAGCTGGGACAT',
 'timecourse_AGCCTTGTGCATCCCT',
 'timecourse_CAATACCTGCTTGCTC',
 'timecourse_ATAATGCGATTATTGC',
 'timecourse_AACCCTCACGGTAACT',
 'timeco

In [7]:
adata.obs
adata.obs_names.copy(deep=True)

Index(['timecourse_GCGGTTGGTAACGTGC', 'timecourse_AAGCCTCCAATCCTGA',
       'timecourse_GGCTGTCAGTTGTCCC', 'timecourse_AGAGATTAGGGTCTAT',
       'timecourse_TCGTTATTCGCCTAAG', 'timecourse_CCTTATGTCCCTCAAC',
       'timecourse_TTTACGCGTTAGCTGA', 'timecourse_CCGCTAGCAGTAATAG',
       'timecourse_GACCTGCAGTGTGATC', 'timecourse_TTCCCACAGTGACCTG',
       ...
       'timecourse_AACCTCACAGCCTAAC', 'timecourse_GTAAGGTCAATTTGGT',
       'timecourse_TTTGACTTCAGGATGA', 'timecourse_ATGGTGCGTTGAAGCC',
       'timecourse_GCAGCAACAAGCGATG', 'timecourse_GAGTAACCATCCTAGA',
       'timecourse_TCTAACCGTTGGTTCT', 'timecourse_CATAATCCATGTCGCG',
       'timecourse_TTTGCGACAGTTATCG', 'timecourse_CACAAGCGTACCAGGT'],
      dtype='object', length=757)

In [8]:
# check if there are common barcodes
list(set(adata.obs_names.copy(deep=True)) & set(list(cistopic_obj.cell_names.copy())))


[]

In [9]:
# maybe select the atac barcodes as defaults for adata
list(set(adata.obs["barcode"]) & set(list(cistopic_obj.cell_names.copy())))

l_bcs = []

for i in range(len(adata.obs_names)):
     l_bc = "timecourse_" + adata.obs["barcode"][i]
     l_bcs.append(l_bc)
     
adata.obs["long_barcode"] = l_bcs
adata.obs_names = list(adata.obs["long_barcode"])


In [10]:
# do the same for the cistopic object
cistopic_obj.cell_data["long_barcode"] = cistopic_obj.cell_names

In [11]:

# check if there are common barcodes
len(list(set(adata.obs_names.copy(deep=True)) & set(list(cistopic_obj.cell_names.copy())))) #128
#len(list(set(adata.obs_names.copy(deep=True)))) #757
#len(set(list(cistopic_obj.cell_names.copy())))  #128


128

In [12]:
cistopic_obj.selected_model.cell_topic.columns.values

array(['timecourse_ATGATGACTAATGTGA', 'timecourse_ACCTCATGAACGGTTG',
       'timecourse_GGCTAGCACCCTCACG', 'timecourse_ATGCTTATGCCCGTAG',
       'timecourse_ACAGCACGAGCGCTAA', 'timecourse_CGGGATTACATAGCTG',
       'timecourse_AACGGTTACCGCTAAG', 'timecourse_ATGCTTTACGAGGGAT',
       'timecourse_AGGCCCAACCTCCTCT', 'timecourse_ATGATGACTGGTCCAC',
       'timecourse_CCAAGGTACTCAATCC', 'timecourse_CAATTTGCTTGTTGCG',
       'timecourse_CATGCTATGGTGCTGT', 'timecourse_ACAGCCAGACCTAGCT',
       'timecourse_ACTAGGATGTGAGTAA', 'timecourse_ACAAACACTTCGCTCC',
       'timecourse_ACAATAGCTTAGCTGC', 'timecourse_AAGGTTTGACAGGCTG',
       'timecourse_AATGAGGGACAAGAAA', 'timecourse_CGATCCTGACTAGTTT',
       'timecourse_ACGGCTCCTAGTAGCG', 'timecourse_ACATTAGGAAGTAAGG',
       'timecourse_AAGCCTAACTTTGTGC', 'timecourse_GAGCCTGACCTAGCTT',
       'timecourse_AACGTAACTGTGACCC', 'timecourse_AGGTTACACGTTTACT',
       'timecourse_AAGGAAGCTGGGACAT', 'timecourse_AGCCTTGTGCATCCCT',
       'timecourse_CAATACCTGCTTGCT

In [13]:
from pycisTopic.cistopic_class import *
from pycisTopic.diff_features import *
common_cells = list(set(adata.obs_names.copy(deep=True)) & set(list(cistopic_obj.cell_names.copy())))
impute_accessibility(cistopic_obj, selected_cells=common_cells)


2023-05-09 15:14:54,349 cisTopic     INFO     Imputing drop-outs
2023-05-09 15:14:54,502 cisTopic     INFO     Scaling
2023-05-09 15:14:54,543 cisTopic     INFO     Keep non zero rows
2023-05-09 15:14:54,611 cisTopic     INFO     Imputed accessibility sparsity: 0.0
2023-05-09 15:14:54,612 cisTopic     INFO     Create CistopicImputedFeatures object
2023-05-09 15:14:54,613 cisTopic     INFO     Done!


## Create SCENIC+ object

In [14]:
# Create the Scenic+ object
from scenicplus.scenicplus_class import create_SCENICPLUS_object
import numpy as np
scplus_obj = create_SCENICPLUS_object(
    GEX_anndata = adata.raw.to_adata(),
    cisTopic_obj = cistopic_obj,
    menr = menr,
    bc_transform_func = None, #lambda x: f'{x}_timecourse' #None, #function to convert scATAC-seq barcodes to scRNA-seq ones
)
scplus_obj.X_EXP = np.array(scplus_obj.X_EXP.todense())
scplus_obj

2023-05-09 15:15:43,150 cisTopic     INFO     Imputing drop-outs
2023-05-09 15:15:43,300 cisTopic     INFO     Scaling
2023-05-09 15:15:43,340 cisTopic     INFO     Keep non zero rows
2023-05-09 15:15:43,407 cisTopic     INFO     Imputed accessibility sparsity: 0.0
2023-05-09 15:15:43,408 cisTopic     INFO     Create CistopicImputedFeatures object
2023-05-09 15:15:43,409 cisTopic     INFO     Done!


SCENIC+ object with n_cells x n_genes = 128 x 27087 and n_cells x n_regions = 128 x 118093
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'features'
	metadata_cell:'GEX_orig.ident', 'GEX_nCount_RNA', 'GEX_nFeature_RNA', 'GEX_percent.mt', 'GEX_percent.ribo', 'GEX_nCount_SCT', 'GEX_nFeature_SCT', 'GEX_SCT_snn_res.0.5', 'GEX_seurat_clusters', 'GEX_pANN_0.25_0.005_794', 'GEX_DF.classifications_0.25_0.005_794', 'GEX_doubletClass', 'GEX_bc', 'GEX_nCount_ATAC', 'GEX_nFeature_ATAC', 'GEX_SCT.weight', 'GEX_ATAC.weight', 'GEX_wsnn_res.0.1', 'GEX_wsnn_res.0.25', 'GEX_wsnn_res.0.5', 'GEX_wsnn_res.0.75', 'GEX_wsnn_res.1', 'GEX_wsnn_res.2', 'GEX_wsnn_res.3', 'GEX_wsnn_res.4', 'GEX_wsnn_res.5', 'GEX_wsnn_res.6', 'GEX_wsnn_res.7', 'GEX_wsnn_res.8', 'GEX_wsnn_res.9', 'GEX_wsnn_res.10', 'GEX_wsnn_res.12', 'GEX_wsnn_res.14', 'GEX_wsnn_res.16', 'GEX_wsnn_res.18', 'GEX_wsnn_res.20', 'GEX_SCT_nn_re

In [15]:
# Select the optimal gene names host
ensembl_version_dict = {'105': 'http://www.ensembl.org',
                        '104': 'http://may2021.archive.ensembl.org/',
                        '103': 'http://feb2021.archive.ensembl.org/',
                        '102': 'http://nov2020.archive.ensembl.org/',
                        '101': 'http://aug2020.archive.ensembl.org/',
                        '100': 'http://apr2020.archive.ensembl.org/',
                        '99': 'http://jan2020.archive.ensembl.org/',
                        '98': 'http://sep2019.archive.ensembl.org/',
                        '97': 'http://jul2019.archive.ensembl.org/',
                        '96': 'http://apr2019.archive.ensembl.org/',
                        '95': 'http://jan2019.archive.ensembl.org/',
                        '94': 'http://oct2018.archive.ensembl.org/',
                        '93': 'http://jul2018.archive.ensembl.org/',
                        '92': 'http://apr2018.archive.ensembl.org/',
                        '91': 'http://dec2017.archive.ensembl.org/',
                        '90': 'http://aug2017.archive.ensembl.org/',
                        '89': 'http://may2017.archive.ensembl.org/',
                        '88': 'http://mar2017.archive.ensembl.org/',
                        '87': 'http://dec2016.archive.ensembl.org/',
                        '86': 'http://oct2016.archive.ensembl.org/',
                        '80': 'http://may2015.archive.ensembl.org/',
                        '77': 'http://oct2014.archive.ensembl.org/',
                        '75': 'http://feb2014.archive.ensembl.org/',
                        '54': 'http://may2009.archive.ensembl.org/'}

import pybiomart as pbm
def test_ensembl_host(scplus_obj, host, species):
    dataset = pbm.Dataset(name=species+'_gene_ensembl',  host=host)
    annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
    annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    annot['Chromosome'] = annot['Chromosome'].astype('str')
    filter = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
    annot = annot[~filter]
    annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    gene_names_release = set(annot['Gene'].tolist())
    ov=len([x for x in scplus_obj.gene_names if x in gene_names_release])
    print('Genes recovered: ' + str(ov) + ' out of ' + str(len(scplus_obj.gene_names)))
    return ov

n_overlap = {}
for version in ensembl_version_dict.keys():
    print(f'host: {version}')
    try:
        n_overlap[version] =  test_ensembl_host(scplus_obj, ensembl_version_dict[version], 'hsapiens')
    except:
        print('Host not reachable')
v = sorted(n_overlap.items(), key=lambda item: item[1], reverse=True)[0][0]
print(f"version: {v} has the largest overlap, use {ensembl_version_dict[v]} as biomart host")

host: 105
Genes recovered: 0 out of 27087
host: 104
Genes recovered: 0 out of 27087
host: 103
Genes recovered: 0 out of 27087
host: 102
Genes recovered: 0 out of 27087
host: 101
Genes recovered: 0 out of 27087
host: 100
Genes recovered: 0 out of 27087
host: 99
Genes recovered: 0 out of 27087
host: 98
Genes recovered: 0 out of 27087
host: 97
Genes recovered: 0 out of 27087
host: 96
Genes recovered: 0 out of 27087
host: 95
Genes recovered: 0 out of 27087
host: 94
Genes recovered: 0 out of 27087
host: 93
Genes recovered: 0 out of 27087
host: 92
Genes recovered: 0 out of 27087
host: 91
Host not reachable
host: 90
Host not reachable
host: 89
Host not reachable
host: 88
Host not reachable
host: 87
Host not reachable
host: 86
Host not reachable
host: 80
Genes recovered: 0 out of 27087
host: 77
Genes recovered: 0 out of 27087
host: 75
Host not reachable
host: 54
Host not reachable
version: 105 has the largest overlap, use http://www.ensembl.org as biomart host


In [16]:
# Choose the best host
biomart_host = "http://sep2019.archive.ensembl.org/"

In [17]:
# Before running  also download a list of known human TFs from the human transcription factors database
!wget -O /g/scb/zaugg/deuner/SCENIC+/inputdata/utoronto_human_tfs_v_1.01.txt  http://humantfs.ccbr.utoronto.ca/download/v_1.01/TF_names_v_1.01.txt


--2023-05-09 15:16:04--  http://humantfs.ccbr.utoronto.ca/download/v_1.01/TF_names_v_1.01.txt
Resolving humantfs.ccbr.utoronto.ca (humantfs.ccbr.utoronto.ca)... 142.150.52.218
Connecting to humantfs.ccbr.utoronto.ca (humantfs.ccbr.utoronto.ca)|142.150.52.218|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11838 (12K) [text/plain]
Saving to: ‘/g/scb/zaugg/deuner/SCENIC+/inputdata/utoronto_human_tfs_v_1.01.txt’

/g/scb/zaugg/deuner 100%[===================>]  11.56K  47.9KB/s    in 0.2s    

2023-05-09 15:16:05 (47.9 KB/s) - ‘/g/scb/zaugg/deuner/SCENIC+/inputdata/utoronto_human_tfs_v_1.01.txt’ saved [11838/11838]



In [18]:
# Also download a the program bedToBigBed this will be used to generate files which can be uploaded to the UCSC genome browser
!wget -O /g/scb/zaugg/deuner/SCENIC+/inputdata/bedToBigBed http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedToBigBed
!chmod +x /g/scb/zaugg/deuner/SCENIC+/inputdata/bedToBigBed

--2023-05-09 15:16:06--  http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedToBigBed
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9630568 (9.2M)
Saving to: ‘/g/scb/zaugg/deuner/SCENIC+/inputdata/bedToBigBed’

/g/scb/zaugg/deuner 100%[===================>]   9.18M  5.20MB/s    in 1.8s    

2023-05-09 15:16:08 (5.20 MB/s) - ‘/g/scb/zaugg/deuner/SCENIC+/inputdata/bedToBigBed’ saved [9630568/9630568]



In [19]:
#only keep the first two columns of the PCA embedding in order to be able to visualize this in SCope
scplus_obj.dr_cell['GEX_X_pca'] = scplus_obj.dr_cell['GEX_X_pca'].iloc[:, 0:2]
#scplus_obj.dr_cell['GEX_rep'] = scplus_obj.dr_cell['GEX_rep'].iloc[:, 0:2]

In [20]:
#ray.shutdown()
#ray.init()

In [21]:
# Run the analysis
from scenicplus.wrappers.run_scenicplus import run_scenicplus
#try:
run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['GEX_celltype'],
        species = 'hsapiens',
        assembly = 'hg38',
        tf_file = '/g/scb/zaugg/deuner/SCENIC+/inputdata/utoronto_human_tfs_v_1.01.txt',
        save_path = os.path.join(tmp_dir, 'scenicplus'),
        biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = '/g/scb/zaugg/deuner/SCENIC+/inputdata',
        n_cpu = 1,
        _temp_dir = None)#os.path.join(tmp_dir, 'ray_spill'))
#except Exception as e:
#    #in case of failure, still save the object
#    dill.dump(scplus_obj, open(os.path.join(out_dir, '/scplus_obj.pkl'), 'wb'), protocol=-1)
#    raise(e)

2023-05-09 15:17:31,197 SCENIC+_wrapper INFO     /g/scb/zaugg/deuner/SCENIC+/tmp/scenicplus folder already exists.
2023-05-09 15:17:31,199 SCENIC+_wrapper INFO     Merging cistromes
2023-05-09 15:18:39,236 SCENIC+_wrapper INFO     Getting search space
2023-05-09 15:18:41,387 R2G          INFO     Downloading gene annotation from biomart dataset: hsapiens_gene_ensembl
2023-05-09 15:19:33,488 R2G          INFO     Downloading chromosome sizes from: http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
2023-05-09 15:19:35,288 R2G          INFO     Extending promoter annotation to 10 bp upstream and 10 downstream
2023-05-09 15:19:39,202 R2G          INFO     Extending search space to:
            						150000 bp downstream of the end of the gene.
            						150000 bp upstream of the start of the gene.
2023-05-09 15:20:01,125 R2G          INFO     Intersecting with regions.


join: Strand data from other will be added as strand data to self.
If this is undesired use the flag apply_strand_suffix=False.
To turn off the warning set apply_strand_suffix to True or False.


2023-05-09 15:20:02,196 R2G          INFO     Calculating distances from region to gene
2023-05-09 15:21:19,240 R2G          INFO     Imploding multiple entries per region and gene
2023-05-09 15:24:17,687 R2G          INFO     Done!
2023-05-09 15:24:17,966 SCENIC+_wrapper INFO     Inferring region to gene relationships
2023-05-09 15:24:18,077 R2G          INFO     Calculating region to gene importances, using GBM method


2023-05-09 15:24:53,582	ERROR services.py:1195 -- Failed to start the dashboard: Failed to start the dashboard
Failed to read dashboard log: [Errno 2] No such file or directory: '/tmp/ray/session_2023-05-09_15-24-18_544842_241410/logs/dashboard.log'
2023-05-09 15:24:53,584	ERROR services.py:1196 -- Failed to start the dashboard
Failed to read dashboard log: [Errno 2] No such file or directory: '/tmp/ray/session_2023-05-09_15-24-18_544842_241410/logs/dashboard.log'
Traceback (most recent call last):
  File "/g/scb/zaugg/deuner/miniconda3/envs/scenicplus/lib/python3.8/site-packages/ray/_private/services.py", line 1167, in start_api_server
    with open(dashboard_log, "rb") as f:
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/ray/session_2023-05-09_15-24-18_544842_241410/logs/dashboard.log'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/g/scb/zaugg/deuner/miniconda3/envs/scenicplus/lib/python3.8/site-package

2023-05-09 15:25:03,316 R2G          INFO     Took 45.2376868724823 seconds
2023-05-09 15:25:03,317 R2G          INFO     Calculating region to gene correlation, using SR method


2023-05-09 15:25:26,071	ERROR services.py:1195 -- Failed to start the dashboard: Failed to start the dashboard
 The last 10 lines of /tmp/ray/session_2023-05-09_15-25-03_792180_241410/logs/dashboard.log:
  File "/g/scb/zaugg/deuner/miniconda3/envs/scenicplus/lib/python3.8/socketserver.py", line 452, in __init__
    self.server_bind()
  File "/g/scb/zaugg/deuner/miniconda3/envs/scenicplus/lib/python3.8/wsgiref/simple_server.py", line 50, in server_bind
    HTTPServer.server_bind(self)
  File "/g/scb/zaugg/deuner/miniconda3/envs/scenicplus/lib/python3.8/http/server.py", line 138, in server_bind
    socketserver.TCPServer.server_bind(self)
  File "/g/scb/zaugg/deuner/miniconda3/envs/scenicplus/lib/python3.8/socketserver.py", line 466, in server_bind
    self.socket.bind(self.server_address)
OSError: [Errno 98] Address already in use
2023-05-09 15:25:26,073	ERROR services.py:1196 -- Failed to start the dashboard
 The last 10 lines of /tmp/ray/session_2023-05-09_15-25-03_792180_241410/logs/

2023-05-09 15:25:34,195 R2G          INFO     Took 30.87720561027527 seconds
An error occured!
2023-05-09 15:25:34,211 SCENIC+_wrapper INFO     Inferring TF to gene relationships


ValueError: Intersection of gene_names and tf_names is empty.

## Downstream Analysis

### Simplifying and filtering SCENIC+ output

In [ ]:
from scenicplus.preprocessing.filtering import apply_std_filtering_to_eRegulons
apply_std_filtering_to_eRegulons(scplus_obj)

In [ ]:
scplus_obj.uns['eRegulon_metadata_filtered'].head()

### eRegulon enrichment scores

In [ ]:
from scenicplus.eregulon_enrichment import score_eRegulons
region_ranking = dill.load(open(os.path.join(out_dir, 'scenicplus/region_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
gene_ranking = dill.load(open(os.path.join(out_dir, 'scenicplus/gene_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
score_eRegulons(scplus_obj,
                ranking = region_ranking,
                eRegulon_signatures_key = 'eRegulon_signatures_filtered',
                key_added = 'eRegulon_AUC_filtered',
                enrichment_type= 'region',
                auc_threshold = 0.05,
                normalize = False,
                n_cpu = 5)
score_eRegulons(scplus_obj,
                gene_ranking,
                eRegulon_signatures_key = 'eRegulon_signatures_filtered',
                key_added = 'eRegulon_AUC_filtered',
                enrichment_type = 'gene',
                auc_threshold = 0.05,
                normalize= False,
                n_cpu = 5)

### eRegulon dimensionality reduction

In [ ]:
from scenicplus.dimensionality_reduction import run_eRegulons_tsne, run_eRegulons_umap
run_eRegulons_umap(
    scplus_obj = scplus_obj,
    auc_key = 'eRegulon_AUC_filtered',
    reduction_name = 'eRegulons_UMAP', #overwrite previously calculated UMAP
)
run_eRegulons_tsne(
    scplus_obj = scplus_obj,
    auc_key = 'eRegulon_AUC_filtered',
    reduction_name = 'eRegulons_tSNE', #overwrite previously calculated tSNE
)

In [ ]:
# Visualize it
from scenicplus.dimensionality_reduction import plot_metadata_given_ax
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#specify color_dictionary

color_dict = {
    'neuron': "#065143",
    'hiPSC': "#70B77E",
    'microglia': "#E0A890",
    'diff.state': "#053C5E" 
}

fig, axs = plt.subplots(ncols=2, figsize = (16, 8))
plot_metadata_given_ax(
    scplus_obj=scplus_obj,
    ax = axs[0],
    reduction_name = 'eRegulons_UMAP',
    variable = 'GEX_celltype', #note the GEX_ prefix, this metadata originated from the gene expression metadata (on which we did the cell type annotation before)
    color_dictionary={'GEX_celltype': color_dict}
)
plot_metadata_given_ax(
    scplus_obj=scplus_obj,
    ax = axs[1],
    reduction_name = 'eRegulons_tSNE',
    variable = 'GEX_celltype', #note the GEX_ prefix, this metadata originated from the gene expression metadata (on which we did the cell type annotation before)
    color_dictionary={'GEX_celltype': color_dict}
)
fig.tight_layout()
sns.despine(ax = axs[0]) #remove top and right edge of axis border
sns.despine(ax = axs[1]) #remove top and right edge of axis border
plt.show()

### plot the activity / expression of an eRegulon on the dimensionality reduction

In [ ]:
from scenicplus.dimensionality_reduction import plot_eRegulon
plot_eRegulon(
    scplus_obj = scplus_obj,
    reduction_name = 'eRegulons_tSNE',
    selected_regulons = ['POU4F3', 'KLF12', 'POU4F1', 'CUX2', 'ONECUT3'],
    scale = True,
    auc_key = 'eRegulon_AUC_filtered')

### dotplot-heatmap


In [ ]:
# We first generate pseudobulk gene expression and region accessibility data, per celltype, to limit the amount of noise for the correlation calculation.
from scenicplus.cistromes import TF_cistrome_correlation, generate_pseudobulks

generate_pseudobulks(
        scplus_obj = scplus_obj,
        variable = 'GEX_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_key = 'Gene_based')
generate_pseudobulks(
        scplus_obj = scplus_obj,
        variable = 'GEX_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_key = 'Region_based')

TF_cistrome_correlation(
            scplus_obj,
            use_pseudobulk = True,
            variable = 'GEX_celltype',
            auc_key = 'eRegulon_AUC_filtered',
            signature_key = 'Gene_based',
            out_key = 'filtered_gene_based')
TF_cistrome_correlation(
            scplus_obj,
            use_pseudobulk = True,
            variable = 'GEX_celltype',
            auc_key = 'eRegulon_AUC_filtered',
            signature_key = 'Region_based',
            out_key = 'filtered_region_based')

In [ ]:
scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based'].head()

In [ ]:
# Let's visualize these correlations in a scatter plot and select eRegulons for which the correlaiton coefficient is above 0.70 or below -0.75
import numpy as np
n_targets = [int(x.split('(')[1].replace('r)', '')) for x in scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Cistrome']]
rho = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'].to_list()
adj_pval = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Adjusted_p-value'].to_list()

thresholds = {
        'rho': [-0.75, 0.70],
        'n_targets': 0
}
import seaborn as sns
fig, ax = plt.subplots(figsize = (10, 5))
sc = ax.scatter(rho, n_targets, c = -np.log10(adj_pval), s = 5)
ax.set_xlabel('Correlation coefficient')
ax.set_ylabel('nr. target regions')
#ax.hlines(y = thresholds['n_targets'], xmin = min(rho), xmax = max(rho), color = 'black', ls = 'dashed', lw = 1)
ax.vlines(x = thresholds['rho'], ymin = 0, ymax = max(n_targets), color = 'black', ls = 'dashed', lw = 1)
ax.text(x = thresholds['rho'][0], y = max(n_targets), s = str(thresholds['rho'][0]))
ax.text(x = thresholds['rho'][1], y = max(n_targets), s = str(thresholds['rho'][1]))
sns.despine(ax = ax)
fig.colorbar(sc, label = '-log10(adjusted_pvalue)', ax = ax)
plt.show()

In [ ]:
selected_cistromes = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based'].loc[
        np.logical_or(
                scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'] > thresholds['rho'][1],
                scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'] < thresholds['rho'][0]
        )]['Cistrome'].to_list()
selected_eRegulons = [x.split('_(')[0] for x in selected_cistromes]
selected_eRegulons_gene_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Gene_based'].keys()
        if x.split('_(')[0] in selected_eRegulons]
selected_eRegulons_region_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Region_based'].keys()
        if x.split('_(')[0] in selected_eRegulons]
#save the results in the scenicplus object
scplus_obj.uns['selected_eRegulon'] = {'Gene_based': selected_eRegulons_gene_sig, 'Region_based': selected_eRegulons_region_sig}
print(f'selected: {len(selected_eRegulons_gene_sig)} eRegulons')

In [ ]:
# Save these changes we have made to the scenicplus_obj
dill.dump(scplus_obj, open(os.path.join(out_dir, 'scenicplus/scplus_obj.pkl'), 'wb'), protocol=-1)

In [ ]:
# Plot the heatmap-dotplot
from scenicplus.plotting.dotplot import heatmap_dotplot
heatmap_dotplot(
        scplus_obj = scplus_obj,
        size_matrix = scplus_obj.uns['eRegulon_AUC_filtered']['Region_based'], #specify what to plot as dot sizes, target region enrichment in this case
        color_matrix = scplus_obj.to_df('EXP'), #specify  what to plot as colors, TF expression in this case
        scale_size_matrix = True,
        scale_color_matrix = True,
        group_variable = 'GEX_celltype',
        subset_eRegulons = scplus_obj.uns['selected_eRegulon']['Gene_based'],
        figsize = (5, 20),
        orientation = 'vertical')

### Overlap of predicted target regions

In [ ]:
# calculate the RSS for the target regions of the selected eRegulons.
from scenicplus.RSS import *
regulon_specificity_scores(
        scplus_obj,
        variable = 'GEX_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_keys = ['Region_based'],
        selected_regulons = [x for x in scplus_obj.uns['selected_eRegulon']['Region_based'] if '-' not in x],
        out_key_suffix = '_filtered')

In [ ]:
# visualize the RSS values using a scatter plot
plot_rss(scplus_obj, 'GEX_celltype_filtered', num_columns=2, top_n=10, figsize = (5, 10))

In [ ]:
# select the top 10 eRegulons per cell type
flat_list = lambda t: [item for sublist in t for item in sublist]
selected_markers = list(set(flat_list(
    [scplus_obj.uns['RSS']['GEX_celltype_filtered'].loc[celltype].sort_values(ascending = False).head(10).index.to_list()
    for celltype in scplus_obj.uns['RSS']['GEX_celltype_filtered'].index])))

In [ ]:
from scenicplus.plotting.correlation_plot import *

region_intersetc_data, Z = jaccard_heatmap(
        scplus_obj,
        method = 'intersect',
        gene_or_region_based = 'Region_based',
        use_plotly = False,
        selected_regulons = selected_markers,
        signature_key = 'eRegulon_signatures_filtered',
        figsize = (10, 10), return_data = True, vmax = 0.5, cmap = 'plasma')

### Plotting a Network

In [ ]:
from pycisTopic.diff_features import find_highly_variable_features
hvr = find_highly_variable_features(scplus_obj.to_df('ACC').loc[list(set(scplus_obj.uns['eRegulon_metadata_filtered']['Region']))], n_top_features=1000, plot = False)
hvg = find_highly_variable_features(scplus_obj.to_df('EXP')[list(set(scplus_obj.uns['eRegulon_metadata_filtered']['Gene']))].T, n_top_features=1000, plot = False)

In [ ]:
from scenicplus.networks import create_nx_tables, create_nx_graph, plot_networkx, export_to_cytoscape
nx_tables = create_nx_tables(
    scplus_obj = scplus_obj,
    eRegulon_metadata_key ='eRegulon_metadata_filtered',
    subset_eRegulons = ['PAX5', 'EBF1', 'POU2AF1'],
    subset_regions = hvr,
    subset_genes = hvg,
    add_differential_gene_expression = True,
    add_differential_region_accessibility = True,
    differential_variable = ['GEX_celltype'])

In [ ]:
G, pos, edge_tables, node_tables = create_nx_graph(nx_tables,
                   use_edge_tables = ['TF2R','R2G'],
                   color_edge_by = {'TF2R': {'variable' : 'TF', 'category_color' : {'PAX5': 'Orange', 'EBF1': 'Purple', 'POU2AF1': 'Red'}},
                                    'R2G': {'variable' : 'R2G_rho', 'continuous_color' : 'viridis', 'v_min': -1, 'v_max': 1}},
                   transparency_edge_by =  {'R2G': {'variable' : 'R2G_importance', 'min_alpha': 0.1, 'v_min': 0}},
                   width_edge_by = {'R2G': {'variable' : 'R2G_importance', 'max_size' :  1.5, 'min_size' : 1}},
                   color_node_by = {'TF': {'variable': 'TF', 'category_color' : {'PAX5': 'Orange', 'EBF1': 'Purple', 'POU2AF1': 'Red'}},
                                    'Gene': {'variable': 'GEX_celltype_Log2FC_B_cells_1', 'continuous_color' : 'bwr'},
                                    'Region': {'variable': 'GEX_celltype_Log2FC_B_cells_1', 'continuous_color' : 'viridis'}},
                   transparency_node_by =  {'Region': {'variable' : 'GEX_celltype_Log2FC_B_cells_1', 'min_alpha': 0.1},
                                    'Gene': {'variable' : 'GEX_celltype_Log2FC_B_cells_1', 'min_alpha': 0.1}},
                   size_node_by = {'TF': {'variable': 'fixed_size', 'fixed_size': 30},
                                    'Gene': {'variable': 'fixed_size', 'fixed_size': 15},
                                    'Region': {'variable': 'fixed_size', 'fixed_size': 10}},
                   shape_node_by = {'TF': {'variable': 'fixed_shape', 'fixed_shape': 'ellipse'},
                                    'Gene': {'variable': 'fixed_shape', 'fixed_shape': 'ellipse'},
                                    'Region': {'variable': 'fixed_shape', 'fixed_shape': 'diamond'}},
                   label_size_by = {'TF': {'variable': 'fixed_label_size', 'fixed_label_size': 20.0},
                                    'Gene': {'variable': 'fixed_label_size', 'fixed_label_size': 10.0},
                                    'Region': {'variable': 'fixed_label_size', 'fixed_label_size': 0.0}},
                   layout='kamada_kawai_layout',
                   scale_position_by=250)

In [ ]:
plt.figure(figsize=(10,10))
plot_networkx(G, pos)

In [ ]:
export_to_cytoscape(G, pos, out_file = os.path.join(out_dir, 'scenicplus/network_timecourse.cys'))